In [1]:
import os
import pickle
import numpy as np
import cv2
from scipy.spatial.distance import euclidean


# Decide how to rank by similarity(distance)
ASC = 'ascend'
DESC = 'descend'

# Folder of pre-trained feature files
FEATURE_PATH = 'features'

# Pre-trained Feature files
COLOR_FEATURE_FILE = 'rch.p'
TEXTURE_FEATURE_FILE = 'gabor.p'
LOCAL_FEATURE_FILE = 'sift.p'

# Method to compare histogram
HISTCMP_METHODS = {
	'Correlation': (cv2.HISTCMP_CORREL, DESC),
	'Chi-Squared': (cv2.HISTCMP_CHISQR, ASC),
	'Intersection': (cv2.HISTCMP_INTERSECT, DESC),
	'Hellinger': (cv2.HISTCMP_BHATTACHARYYA, ASC),
}


In [2]:
def image_extract(feature_file, similarity_func, orderby):
    # feature_file: File of pre-trained feature
    # similarity_func: Function to calculate similarity(distance)
    # orderby: Decide how to rank(ASC or DESC)

    # load features from feature file
    with open(os.path.join(FEATURE_PATH, feature_file), 'rb') as fp:
        features = pickle.load(fp)
        
    # leave-one-out image extracting
    # map_across_dataset: Average of MAP of all categories
    # map_rank: Rank of Category by MAP per category
    map_across_dataset, map_rank = leave_one_out(features, similarity_func, orderby)

    return (map_across_dataset, map_rank[0], map_rank[1], map_rank[-1], map_rank[-2])


In [3]:
def leave_one_out(features, similarity_func, orderby):
    map_per_category = dict()
    map_across_dataset = 0

    # calculate AP for each query image
    # calculate MAP for each category
    for category in features.keys():
        map_per_category[category] = 0
        for index in features[category].keys():
            ap = get_ap(category, index, features, similarity_func, orderby)
            map_per_category[category] += ap
        map_per_category[category] = map_per_category[category] / len(features[category])
        map_across_dataset += map_per_category[category]

    # calculate average of MAP of all categories
    map_across_dataset = map_across_dataset / len(features)
    
    # rank category by MAP per category
    map_rank = sorted([(v, k) for (k, v) in map_per_category.items()], reverse=True)

    return map_across_dataset, map_rank


In [4]:
def get_ap(target_category, target_index, features, similarity_func, orderby):
    # target_category: category of query image
    # target_index: index of query image

    dist_rank = []
    result_rank = []
    query = features[target_category][target_index]
    
    # use similarity_func to calculate similarity(distance) between query image and other images
    for category in features.keys():
        for index in features[category].keys():
            if category == target_category and index == target_index:
                continue
            dist = similarity_func(query, features[category][index])

            # insert category and index of image into rank list in the ascend order of similarity(distance)
            insert_index = len(dist_rank)
            for i, d in enumerate(dist_rank):
                if dist <= d:
                    insert_index = i
                    break

            dist_rank.insert(insert_index, dist)
            result_rank.insert(insert_index, (category, index))

    # if order by descend, reverse the rank list
    if orderby == DESC:
        dist_rank.reverse()
        result_rank.reverse()

    # calculate AP of the query image from rank list
    ap = rank2ap(result_rank, target_category)

    return ap


In [5]:
def rank2ap(result_rank, target_category):
    retrieve_num = 0
    retrieve_precision = []
    for rank, (category, index) in enumerate(result_rank):
        if category == target_category:
            retrieve_num += 1
            retrieve_precision.append(retrieve_num / (rank + 1))

    return np.mean(retrieve_precision)


In [6]:
# COLOR FEATURES:
feature_file = COLOR_FEATURE_FILE
method, orderby = HISTCMP_METHODS['Intersection']
similarity_func = lambda query, feature: cv2.compareHist(query, feature, method)

color_result = image_extract(feature_file, similarity_func, orderby)
# color_result: map, best_first, best_second, worst_first, worst_second


In [7]:
# TEXTURE FEATURES:
feature_file = TEXTURE_FEATURE_FILE
similarity_func = lambda query, feature: euclidean(query, feature)


texture_result = image_extract(feature_file, similarity_func, ASC)
# texture_result: map, best_first, best_second, worst_first, worst_second

In [8]:
# LOCAL FEATURES:
feature_file = LOCAL_FEATURE_FILE
method, orderby = HISTCMP_METHODS['Intersection']
similarity_func = lambda query, feature: cv2.compareHist(query, feature, method)

local_result = image_extract(feature_file, similarity_func, orderby)
# local)result: map, best_first, best_second, worst_first, worst_second

| Methods v.s. Categories | MAP | Best Two Categories | Worst Two Categories |
| :---: | :---: | :---: | :---: |
| Color<br>(Regional Color Histogram) | {{color_result[0]}} | 1. {{color_result[1][1]}}: {{color_result[1][0]}}<br> 2. {{color_result[2][1]}}: {{color_result[2][0]}} | 1. {{color_result[3][1]}}: {{color_result[3][0]}}<br>2. {{color_result[4][1]}}: {{color_result[4][0]}} |
| Texture<br>(Gabor Filter) | {{texture_result[0]}} | 1. {{texture_result[1][1]}}: {{texture_result[1][0]}}<br> 2. {{texture_result[2][1]}}: {{texture_result[2][0]}} | 1. {{texture_result[3][1]}}: {{texture_result[3][0]}}<br>2. {{texture_result[4][1]}}: {{texture_result[4][0]}} |
| Local<br>(DOG+SIFT+KMeans) | {{local_result[0]}} | 1. {{local_result[1][1]}}: {{local_result[1][0]}}<br> 2. {{local_result[2][1]}}: {{local_result[2][0]}} | 1. {{local_result[3][1]}}: {{local_result[3][0]}}<br>2. {{local_result[4][1]}}: {{local_result[4][0]}} |